In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
import wandb

/data/ephemeral/home/vm/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
    
# config file 들고오기
filename = './configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py'
cfg = Config.fromfile(filename)


root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

# wandb logger hook 추가
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': "object_detection",
                     'entity' : "cv-2",
                     'name' : filename.split('/')[-1]},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)]



cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_r50_fpn_1x_coco'

# change number of classes
if(filename in ['./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py',
                './configs/convnext/cascade_mask_rcnn_convnext-t_p4_w7_fpn_giou_4conv1f_fp16_ms-crop_3x_coco.py']):
    cfg.model.roi_head.bbox_head[0].num_classes=10
    cfg.model.roi_head.bbox_head[1].num_classes=10
    cfg.model.roi_head.bbox_head[2].num_classes=10
elif(filename in ['./configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py',
                     './configs/detr/detr_r50_8x2_150e_coco.py']):
    cfg.model.bbox_head.num_classes = 10
elif(filename in ['./configs/yolox/yolox_tiny_8x8_300e_coco.py']):
    cfg.model.bbox_head.feat_channels=10
elif(filename in ['./configs/yolo/yolov3_d53_320_273e_coco.py']):
    pass
else:
    cfg.model.roi_head.bbox_head.num_classes = 10


#epoch 수 변경
cfg.runner = dict(type='EpochBasedRunner', max_epochs=35)


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [5]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-08 23:35:27,140 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-01-08 23:35:27,142 - mmcv - INFO - load model from: torchvision://resnet50
2024-01-08 23:35:27,142 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-01-08 23:35:27,265 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-01-08 23:35:27,287 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-08 23:35:27,315 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-01-08 23:35:27,322 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [7]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-08 23:35:38,025 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-08 23:35:38,029 - mmdet - INFO - Start running, host: root@instance-5087, work_dir: /data/ephemeral/home/baseline/mmdetection/work_dirs/cascade_rcnn_r50_fpn_1x_coco_resize
2024-01-08 23:35:38,030 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerH

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


2024-01-08 23:37:16,662 - mmdet - WARNING - No meta information found in the runner. 
2024-01-08 23:37:16,664 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2024-01-08 23:37:48,904 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-03, eta: 2:36:53, time: 0.645, data_time: 0.063, memory: 7216, loss_rpn_cls: 0.3651, loss_rpn_bbox: 0.0423, s0.loss_cls: 0.6870, s0.acc: 87.2363, s0.loss_bbox: 0.1017, s1.loss_cls: 0.2726, s1.acc: 89.1152, s1.loss_bbox: 0.0292, s2.loss_cls: 0.1254, s2.acc: 92.6289, s2.loss_bbox: 0.0041, loss: 1.6274, grad_norm: 8.7882
2024-01-08 23:38:17,316 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-03, eta: 2:27:04, time: 0.568, data_time: 0.015, memory: 7216, loss_rpn_cls: 0.1275, loss_rpn_bbox: 0.0396, s0.loss_cls: 0.3198, s0.acc: 93.6934, s0.loss_bbox: 0.1494, s1.loss_cls: 0.0878, s1.acc: 97.1592, s1.loss_bbox: 0.0452, s2.loss_cls: 0.0257,

KeyboardInterrupt: 

10 10 10
